In [253]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import scipy as sp
import math
from prettytable import PrettyTable
import copy

func - функция

grad_func - градиент func

norm - норма вектора

eps_global - точность

b - парааметр для овражной функции

In [254]:
# Основная функция
def func(x, a = 0, grad = np.array([0, 0])):
    #return 211*x[0]**2 - 420*x[0]*x[1] + 211*x[1]**2 - 192*x[0] + 50*x[1] - 25
    return 211*(x[0] - a*grad[0])**2 - \
            420*(x[0] - a*grad[0])*(x[1] - a*grad[1]) + \
            211*(x[1] - a*grad[1])**2 - \
            192*(x[0] - a*grad[0]) + \
            50*(x[1] - a*grad[1]) - 25

def grad_func(x):
    return np.array([
        422*x[0] - 420*x[1] - 192,
        -420*x[0] + 422*x[1] + 50
    ])

def H_func(x):
    return np.array([[422, -420], [-420, 422]])

def H_1_func(x):
    return np.array([[211/842, 105/421], [105/421, 211/842]])

def norm(x):
    y = 0
    for i in x:
        y += i**2
    return math.sqrt(y)

eps_global = 1e-6
b = 10

In [255]:
# Овражная функция
def f_ovrag(x, a = 0, grad = np.array([0, 0])):
    return (x[0] - a*grad[0])**2 + b*(x[1] - a*grad[1])**2

def grad_ovrag(x):
    return np.array([
        2*x[0], 2*b*x[1]
    ])

def H_ovrag(x):
    return np.array([[2, 0], [0, 2*b]])

def H_1_ovrag(x):
    return np.array([[0.5, 0], [0, 1/(2*b)]])

In [256]:
# функция Розенброка
def f_rozenbrok(x, a = 0, grad = np.array([0, 0])):
    return 100*((x[0] - a*grad[0])**2 - (x[1] - a*grad[1]))**2 + ((x[0] - a*grad[0]) - 1)**2
    #return (x[0] - a*grad[0])**2 + b*(x[1] - a*grad[1])**2

def grad_rozenbrok(x):
    return np.array([
        #202*x[0] - 2, -(200*x[1])
        400*(x[0]**3) - 400*x[0]*(x[1]**2) + 2*x[0] - 2,
        -400*(x[0]**2)*x[1] + 400*(x[1]**3)
    ])

def H_rozenbrog(x):
    return np.array([
        [1200*x[0]**2-400*x[1]**2 + 2, -800*x[0]*x[1]], 
        [-800*x[0]*x[1], -400*x[0]**2 + 1200*x[1]**2]
    ])

def H_1_rozenbrog(x):
    return inv(H_rozenbrog(x))
    #return np.array([[1/202, 0], [0, -1/200]])

In [257]:
# функция Химмельблау
def f_himelblay(x, a = 0, grad = np.array([0, 0])):
    return ((x[0] - a*grad[0])**2 + (x[1] - a*grad[1]) - 11)**2 + ((x[0] - a*grad[0]) + (x[1] - a*grad[1])**2 - 7)**2

def grad_himelblay(x):
    return np.array([
        4*(x[0]**3) + 4*x[0]*x[1] - 42*x[0] + 2*x[1]**2 - 14, 
        -26*x[1] + 2*(x[0]**2) - 22 + 4*(x[1]**3) + 4*x[0]*x[1]
    ])

def H_himelblay(x):
    return np.array([
        [12*x[0]**2 + 4*x[1] - 42, 4*x[0] + 4*x[1]], 
        [4*x[0] + 4*x[1], -26 + 12*x[1]**2 + 4*x[0]]
    ])

def H_1_himelblay(x):
    return inv(H_himelblay(x))

In [258]:
# Метод поразрядного поиска
def bitwise(f, x, grad, left, right, e):
    step = (right - left) / 4
    a_last = left
    y_last = f(x, left, grad)
    
    n = 1
    
    even = True

    while step >= e:
        if even:
            while True:
                a = min(a_last + step, right)
                y = f(x, a, grad)
                n += 1
                if y > y_last or a == right:
                    y_last = y
                    a_last = a
                    right = a
                    break
                y_last = y
                a_last = a
        else:
            while True:
                a = max(left, a_last - step)
                y = f(x, a, grad)
                n += 1
                if y > y_last or a == left:
                    y_last = y
                    a_last = a
                    left = a
                    break
                y_last = y
                a_last = a
        
        step /= 4
        even = not even

    return a, n

In [259]:
# Метод Дэвиса-Свенна-Кэмпи
# \param [in] f - функция
# \param [in] a_0 - начальное значение
# return left,right - границы поиска

'''
def DSK(f, x, a_0, grad, e):
    h = 1
    left = a_0
    right = a_0
    f_0 = f(x, a_0, grad)

    a_last = 0

    if (f_0 > f(x, (a_0 + h), grad)):
        right = a_0 + h
        a_last = right
    elif (f_0 > f(x, (a_0 - h), grad)):
        left = a_0 - h
        a_last = left
        h = -h
    else:
        left = a_0 - h
        right = a_0 + h
        return left, right
    
    #print("start")
    #print(left,right)
    
    while(f(x, a_last, grad) < f(x, a_last + h, grad)):
        a_last += h
        h *= 2
        #print(h, left, right, a_last)
        if (abs(h) > 20):
            break
        
    #print(h, left, right, a_last)
    #print()
    if (h < 0):
        return a_last, right
    else:
        return left, a_last
'''

'''
def DSK(f, x, a_0, grad, e):
    h = 0.5
    left = a_0
    right = a_0

    if f(x, a_0, grad) > f(x, (a_0 + h), grad):
        left = a_0
    elif f(x, (a_0 - h), grad) >= f(x, a_0, grad):
        left = a_0 - h
        right = a_0 + h
        return left, right
    else:
        right = a_0
        h = -h

    def xk(k):
        return a_0 + (2 ** (k - 1)) * h

    k = 2
    while True:
        xk0, xk1 = xk(k), xk(k - 1)
        print(xk0, xk1)
        if f(x, xk0, grad) >= f(x, xk1, grad):
            if h < 0:
                left = xk0
            else:
                right = xk0
            break
        else:
            if h > 0:
                left = xk1
            else:
                right = xk1
        k += 1

    return left, right
    '''

'\ndef DSK(f, x, a_0, grad, e):\n    h = 0.5\n    left = a_0\n    right = a_0\n\n    if f(x, a_0, grad) > f(x, (a_0 + h), grad):\n        left = a_0\n    elif f(x, (a_0 - h), grad) >= f(x, a_0, grad):\n        left = a_0 - h\n        right = a_0 + h\n        return left, right\n    else:\n        right = a_0\n        h = -h\n\n    def xk(k):\n        return a_0 + (2 ** (k - 1)) * h\n\n    k = 2\n    while True:\n        xk0, xk1 = xk(k), xk(k - 1)\n        print(xk0, xk1)\n        if f(x, xk0, grad) >= f(x, xk1, grad):\n            if h < 0:\n                left = xk0\n            else:\n                right = xk0\n            break\n        else:\n            if h > 0:\n                left = xk1\n            else:\n                right = xk1\n        k += 1\n\n    return left, right\n    '

## Метод наискорейшего спуска

In [260]:
def fastest_descent(f, f_grad, x_0, e, e_one):
    n_func = 0
    n = 0
    x_last = x_0

    while True:
        p = f_grad(x_last)
        n_func += np.size(x_last)
        #left, right = DSK(f, x_last, 0, p, e)
        a, bitwise_n = bitwise(f, x_last, p, 0, 2, e_one)
        n_func += bitwise_n
        x = x_last - a * p
        x_last = x
        n += 1

        if (norm(p) < e):
            break

        if (n > 100000):
            break

    return x_last, f(x_last), n, n_func

In [261]:
eps_global = 1e-6
e_one = 1e-8
print("Наискорейший спуск\n")
x_min, y_min, count, func_count= fastest_descent(func, grad_func, np.array([4, 4]), eps_global, e_one)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = fastest_descent(f_ovrag, grad_ovrag, np.array([5, 5]), eps_global, e_one)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = fastest_descent(f_himelblay, grad_himelblay, np.array([-5, 0]), eps_global, e_one)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = fastest_descent(f_rozenbrok, grad_rozenbrok, np.array([-1, 1]), eps_global, e_one)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Наискорейший спуск

(6)
x_min = [35.64370579 35.35629423] 
f(x_min) -2562.88836104497 
Иттераций = 1727 
Вызовов функции в совокопности 80387 

(овражная)
x_min = [4.30643408e-09 4.30414900e-09] 
f(x_min) 2.0380236032978714e-16 
Иттераций = 16 
Вызовов функции в совокопности 1077 

(Химельблау)
x_min = [ 3.58442834 -1.84812652] 
f(x_min) 1.7737703919807662e-16 
Иттераций = 15 
Вызовов функции в совокопности 845 

(Розенброка)
x_min = [1.00000006 1.00000006] 
f(x_min) 3.3922768111015547e-13 
Иттераций = 79823 
Вызовов функции в совокопности 2714037 



## Метод сопряженных градиентов

In [262]:
def co_gradients(f, f_grad, x_0, e, e_one):
    n_func = 0
    n = 0
    #m = 2

    x_last = x_0
    p = f_grad(x_last)
    n_func += np.size(x_last)

    while True:

        #left, right = DSK(f, x_last, 0, p, e)
        
        a, bitwise_n = bitwise(f, x_last, p, 0, 2, e_one)
        n_func += bitwise_n
        
        x = x_last - a * p
        #print(x, x_last)

        if (norm(p) < e):
            break

        #if (n % m == 0):
        #    beta = 0
        #else:
        beta = (norm(f_grad(x)) ** 2) / (norm(p) ** 2)
        n_func += np.size(x_last)

        p_new = f_grad(x) + beta * p
        n_func += np.size(x_last)

        #print(x_last, a, p, norm(p), p_new)

        x_last = x
        p = p_new
        n += 1

        if (n > 100000):
            break

    return x_last, f(x_last), n, n_func

In [263]:
eps_global = 1e-3
e_one = 1e-6

#eps_global = 1e-05
#e_one = 0.0001
print("Сопряженных градиентов\n")
x_min, y_min, count, func_count= co_gradients(func, grad_func, np.array([4, 4]), eps_global, e_one)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = co_gradients(f_ovrag, grad_ovrag, np.array([5, 5]), eps_global, e_one)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = co_gradients(f_himelblay, grad_himelblay, np.array([-5, 0]), eps_global, e_one)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = co_gradients(f_rozenbrok, grad_rozenbrok, np.array([-1, 1]), eps_global, e_one)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Сопряженных градиентов

(6)
x_min = [35.64370136 35.35629119] 
f(x_min) -2562.888361044934 
Иттераций = 6 
Вызовов функции в совокопности 318 

(овражная)
x_min = [-1.43368256e-04 -2.73743606e-08] 
f(x_min) 2.055446433668779e-08 
Иттераций = 3 
Вызовов функции в совокопности 230 

(Химельблау)
x_min = [ 3.58443606 -1.84812757] 
f(x_min) 3.078338059653238e-09 
Иттераций = 10 
Вызовов функции в совокопности 529 

(Розенброка)
x_min = [0.99999746 0.99999494] 
f(x_min) 6.448393680672388e-12 
Иттераций = 100001 
Вызовов функции в совокопности 3000064 



## Метод Ньютона

In [264]:
def newton(f, f_grad, f_h_1, x_0, e):
    n_func = 0
    n = 0

    x_last = x_0
    p_last = f_grad(x_last)
    n_func += np.size(x_last)

    while True:

        H_1 = f_h_1(x_last)
        n_func += np.size(H_1)
        
        x = x_last - np.matmul(H_1, p_last)
        n_func += np.size(H_1)

        #print(x, f(x))

        p = f_grad(x)
        n_func += np.size(x)

        x_last = x
        p_last = p
        n += 1

        #print(x, p)

        if (norm(p) < e):
        #if(n > 1000):
            break

    return x_last, f(x_last), n, n_func

In [265]:
eps_global = 1e-6
print("Ньютона\n")
x_min, y_min, count, func_count= newton(func, grad_func, H_1_func, np.array([4, 4]), eps_global)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = newton(f_ovrag, grad_ovrag, H_1_ovrag, np.array([5, 5]), eps_global)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = newton(f_himelblay, grad_himelblay, H_1_himelblay, np.array([-5, -2]), eps_global)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = newton(f_rozenbrok, grad_rozenbrok, H_1_rozenbrog, np.array([-1, 1]), eps_global)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Ньютона

(6)
x_min = [35.64370546 35.35629454] 
f(x_min) -2562.88836104516 
Иттераций = 1 
Вызовов функции в совокопности 12 

(овражная)
x_min = [0. 0.] 
f(x_min) 0.0 
Иттераций = 1 
Вызовов функции в совокопности 12 

(Химельблау)
x_min = [-2.80511809  3.13131252] 
f(x_min) 2.834173752584788e-17 
Иттераций = 12 
Вызовов функции в совокопности 122 

(Розенброка)
x_min = [ 1. -1.] 
f(x_min) 399.9999999999057 
Иттераций = 1 
Вызовов функции в совокопности 12 



## Метод правильного симплекса

In [266]:
def right_simplex(f, x_0, l, sigma, e): # simplex reduction

    def calc_points(x, l):
        n = len(x)
        x_new = np.zeros((n+1, n))
        x_new[0] = x
        for i in range(1, n+1):
            for j in range(n):
                if (i == j+1):
                    x_new[i][j] = x[j] + (math.sqrt(n+1)-1)*l/(n*math.sqrt(2))
                else:
                    x_new[i][j] = x[j] + (math.sqrt(n+1)+n-1)*l/(n*math.sqrt(2))
        return x_new
    
    def reduce_points(x, ii, sigma):
        n = 3
        x_new = np.zeros((n, n-1))
        for i in range(n):
            for j in range(n-1):
                    x_new[i][j] = x[ii][j] + sigma*(x[i][j] - x[ii][j])
        return x_new
    
    def sort_points(f, points):
        sort_point = points
        n = points.shape[0]
        for i in range(n):
            for j in range(n-1):
                if (f(sort_point[j]) < f(sort_point[j+1])):
                    tmp = copy.copy(sort_point[j])
                    sort_point[j] = sort_point[j+1]
                    sort_point[j+1] = tmp
        return sort_point
    
    def get_mirror_point(simplex, ind):
        n = simplex.shape[0] - 1
        x_new = np.zeros((simplex.shape[1]))

        for k in range(simplex.shape[0]):
            if (k != ind):
                x_new += simplex[k]

        x_new = x_new * 2 / n - simplex[ind]
        return x_new

    n_func = 0
    n = 0

    x_last = x_0

    # Build simplex (three point)
    simplex = calc_points(x_last, l)

    #xpoints = np.array(x_0[0])
    #ypoints = np.array(x_0[1])

    while True:
        
        # find f_max from three points (sort on increasing)
        sort_simplex = sort_points(f, simplex)
        n_func += simplex.shape[0]

        # calc x_new
        x_new = x_last


        #print(sort_simplex[0], f(sort_simplex[0]))
        #print(sort_simplex[1], f(sort_simplex[1]))
        #print(sort_simplex[2], f(sort_simplex[2]))
        #print()

        #xpoints = np.append(xpoints, sort_simplex[0][0])
        #ypoints = np.append(ypoints, sort_simplex[0][1])
        #xpoints = np.append(xpoints, sort_simplex[1][0])
        #ypoints = np.append(ypoints, sort_simplex[1][1])
        #xpoints = np.append(xpoints, sort_simplex[2][0])
        #ypoints = np.append(ypoints, sort_simplex[2][1])


        find_lower = False
        for i in range(sort_simplex.shape[0]):
            x_new = get_mirror_point(sort_simplex, i)
            #xpoints = np.append(xpoints, x_new[0])
            #ypoints = np.append(ypoints, x_new[1])

            n_func += 2
            if (f(x_new) < f(sort_simplex[i])):
                x_last = x_new
                simplex[i] = x_last
                find_lower = True
                break

        if (not find_lower): # reduce
            l *= sigma
            #print (sort_simplex, sigma, i, l)
            sort_simplex = reduce_points(sort_simplex, i, sigma)
            #print (sort_simplex, sigma, i, l)
            #print()
            simplex = sort_simplex

        #if (find_min):
        if (l < e or n > 100000):
            #plt.plot(xpoints, ypoints, 'o')
            #plt.show()
            return x_last, f(x_last), n, n_func

        n += 1
            

In [267]:
global b
l = 1e-2
b = 1
sigma = 0.618
eps_global = 1e-6
print("Правильного симплекса с редукцией\n")
x_min, y_min, count, func_count= right_simplex(func, np.array([4, 4]), l, sigma, eps_global)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = right_simplex(f_ovrag, np.array([0.5, 0.5]), l, sigma, eps_global)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = right_simplex(f_himelblay, np.array([-5, -2]), l, sigma, eps_global)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = right_simplex(f_rozenbrok, np.array([-1, 1]), l, sigma, eps_global)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Правильного симплекса с редукцией

(6)
x_min = [35.643703   35.35629237] 
f(x_min) -2562.8883610451708 
Иттераций = 10498 
Вызовов функции в совокопности 53051 

(овражная)
x_min = [0.00032372 0.00032364] 
f(x_min) 2.095345225078792e-07 
Иттераций = 184 
Вызовов функции в совокопности 1005 

(Химельблау)
x_min = [-3.77927486 -3.2831281 ] 
f(x_min) 1.6276919769824512e-07 
Иттераций = 380 
Вызовов функции в совокопности 1985 

(Розенброка)
x_min = [0.99957988 0.99915833] 
f(x_min) 1.7675790407791787e-07 
Иттераций = 15460 
Вызовов функции в совокопности 81749 



In [268]:
def right_simplex_base(f, x_0, l): # simplex reduction

    def calc_points(x, l):
        n = len(x)
        x_new = np.zeros((n+1, n))
        x_new[0] = x
        for i in range(1, n+1):
            for j in range(n):
                if (i == j+1):
                    x_new[i][j] = x[j] + (math.sqrt(n+1)-1)*l/(n*math.sqrt(2))
                else:
                    x_new[i][j] = x[j] + (math.sqrt(n+1)+n-1)*l/(n*math.sqrt(2))
        return x_new
    
    def sort_points(f, points):
        sort_point = points
        n = points.shape[0]
        for i in range(n):
            for j in range(n-1):
                if (f(sort_point[j]) < f(sort_point[j+1])):
                    tmp = copy.copy(sort_point[j])
                    sort_point[j] = sort_point[j+1]
                    sort_point[j+1] = tmp
        return sort_point
    
    def get_mirror_point(simplex, ind):
        n = simplex.shape[0] - 1
        x_new = np.zeros((simplex.shape[1]))

        for k in range(simplex.shape[0]):
            if (k != ind):
                x_new += simplex[k]

        x_new = x_new * 2 / n - simplex[ind]
        return x_new

    n_func = 0
    n = 0

    x_last = x_0

    # Build simplex (three point)
    simplex = calc_points(x_last, l)

    #xpoints = np.array(x_0[0])
    #ypoints = np.array(x_0[1])

    while True:
        
        # find f_max from three points (sort on increasing)
        sort_simplex = sort_points(f, simplex)
        n_func += simplex.shape[0]

        # calc x_new
        x_new = x_last

        #print(sort_simplex[0], f(sort_simplex[0]))
        #print(sort_simplex[1], f(sort_simplex[1]))
        #print(sort_simplex[2], f(sort_simplex[2]))
        #print()

        #xpoints = np.append(xpoints, sort_simplex[0][0])
        #ypoints = np.append(ypoints, sort_simplex[0][1])
        #xpoints = np.append(xpoints, sort_simplex[1][0])
        #ypoints = np.append(ypoints, sort_simplex[1][1])
        #xpoints = np.append(xpoints, sort_simplex[2][0])
        #ypoints = np.append(ypoints, sort_simplex[2][1])

        find_lower = False
        for i in range(sort_simplex.shape[0]):
            x_new = get_mirror_point(sort_simplex, i)
            #xpoints = np.append(xpoints, x_new[0])
            #ypoints = np.append(ypoints, x_new[1])

            n_func += 2
            if (f(x_new) < f(sort_simplex[i])):
                x_last = x_new
                sort_simplex[i] = x_last
                find_lower = True
                break

        simplex = sort_simplex

        if (not find_lower):
            #plt.plot(xpoints, ypoints, 'o')
            #plt.show()
            return x_last, f(x_last), n, n_func

        if (n > 100000):
            return x_last, f(x_last), n, n_func
        
        n += 1
            

In [269]:
global b
b = 1
eps_global = 1e-2
print("Правильного симплекса базовый\n")
x_min, y_min, count, func_count= right_simplex_base(func, np.array([4, 4]), eps_global)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = right_simplex_base(f_ovrag, np.array([0.5, 0.5]), eps_global)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = right_simplex_base(f_himelblay, np.array([-5, -2]), eps_global)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = right_simplex_base(f_rozenbrok, np.array([-1, 1]), eps_global)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Правильного симплекса базовый

(6)
x_min = [35.2718478  34.98193402] 
f(x_min) -2562.6086216233025 
Иттераций = 10169 
Вызовов функции в совокопности 51010 

(овражная)
x_min = [-0.0021454 -0.0021454] 
f(x_min) 9.205458896990295e-06 
Иттераций = 164 
Вызовов функции в совокопности 829 

(Химельблау)
x_min = [-3.78516334 -3.28832135] 
f(x_min) 0.002310026728501199 
Иттераций = 360 
Вызовов функции в совокопности 1809 

(Розенброка)
x_min = [-0.99292893  0.99292893] 
f(x_min) 3.976695268074419 
Иттераций = 1 
Вызовов функции в совокопности 14 



## Метод циклического покоординатного спуска

In [270]:
def cycle_coordinate_fall(f, x_0, e, e_one):
    n_func = 0
    n = 0

    x_last = x_0

    while True:

        x = x_last
        for i in range(x_0.shape[0]):
            p = np.zeros(x_0.shape[0])
            p[i] = 1
            #print(p)
            #left, right = DSK(f, x, 0, p, e)
            a, bitwise_n = bitwise(f, x, p, -2, 2, e_one)
            n_func += bitwise_n
            x = x - a * p
            
        n_func += 2
        if (abs(f(x) - f(x_last)) < e):
            break
        
        x_last = x
        n += 1

        if (n > 10000):
            break


    return x, f(x), n, n_func

In [271]:
eps_global = 1e-6
e_one = 1e-4
print("Циклического покоординатного спуска\n")
x_min, y_min, count, func_count= cycle_coordinate_fall(func, np.array([4, 4]), eps_global, e_one)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = cycle_coordinate_fall(f_ovrag, np.array([0.5, 0.5]), eps_global, e_one)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = cycle_coordinate_fall(f_himelblay, np.array([-5, -2]), eps_global, e_one)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = cycle_coordinate_fall(f_rozenbrok, np.array([-1, 1]), eps_global, e_one)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Циклического покоординатного спуска

(6)
x_min = [35.57861328 35.29125977] 
f(x_min) -2562.8798938393593 
Иттераций = 804 
Вызовов функции в совокопности 56790 

(овражная)
x_min = [-0.00024414 -0.00024414] 
f(x_min) 1.1920928955078125e-07 
Иттераций = 1 
Вызовов функции в совокопности 144 

(Химельблау)
x_min = [-3.77954102 -3.28344727] 
f(x_min) 4.404376348077221e-06 
Иттераций = 4 
Вызовов функции в совокопности 348 

(Розенброка)
x_min = [0.99951172 0.9987793 ] 
f(x_min) 6.2105302731652046e-06 
Иттераций = 1 
Вызовов функции в совокопности 126 



## Метод Хука-Дживса

In [272]:
def hook_jivs(f, gamma, x_0, e):
    a_k = 2
    n_func = 0
    n = 0

    x_last = x_0

    #delta = np.random.rand(x_0.shape[0])
    delta = np.ones(x_0.shape[0])
    #delta = np.full(x_0.shape[0], 2.0)

    while True:

        # search find start
        y_last = f(x_last)
        n_func += 1

        x = x_last
        
        search_find_need = True
        
        while search_find_need:

            for i in range(x.shape[0]):
                p = np.zeros(x.shape[0])
                p[i] = 1
                y = f(x, -delta[i], p)
                n_func += 1

                if (y < y_last):
                    x = x + delta[i] * p
                else:
                    y = f(x, delta[i], p)
                    n_func += 1
                    if (y < y_last):
                        x = x - delta[i] * p

            #print(x, x_last, delta)
            if not((x_last == x).all()):
                #print("not equal")
                #print(x, f(x), x_last, f(x_last), delta)
                search_find_need = False
            else:
                #print("equal")
                #print(x, f(x), x_last, f(x_last), delta)
                if (norm(delta) < e):
                    return x, f(x), n, n_func
                else:
                    delta /= gamma
                        
                    
        # search find end
        

        a_k_local = a_k
        x_new = x_last + a_k_local*(x - x_last)
        while (f(x_new) > f(x_last)):
            a_k_local /= 2
            x_new = x_last + a_k_local*(x - x_last)

        x_last = x_new

        if (n > 100000):
            break

        n += 1


    return x, f(x), n, n_func

In [273]:
eps_global = 1e-6
gamma = 1.618
b = 1
print("Хука-Дживса\n")
x_min, y_min, count, func_count= hook_jivs(func, gamma, np.array([4, 4]), eps_global)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = hook_jivs(f_ovrag, gamma, np.array([2, 2]), eps_global)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = hook_jivs(f_himelblay, gamma, np.array([-5, -2]), eps_global)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = hook_jivs(f_rozenbrok, gamma, np.array([-1, 1]), eps_global)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Хука-Дживса

(6)
x_min = [35.64370348 35.35629255] 
f(x_min) -2562.8883610452067 
Иттераций = 69 
Вызовов функции в совокопности 385 

(овражная)
x_min = [0. 0.] 
f(x_min) 0.0 
Иттераций = 1 
Вызовов функции в совокопности 130 

(Химельблау)
x_min = [-2.80511795  3.13131233] 
f(x_min) 2.007148333330373e-12 
Иттераций = 30 
Вызовов функции в совокопности 250 

(Розенброка)
x_min = [0.99994581 0.99989188] 
f(x_min) 2.9442314611548254e-09 
Иттераций = 1623 
Вызовов функции в совокопности 5837 



## Метод случайного поиска

In [274]:
def random_find(f, alfa, gamma, M, x_0, e):
    n_func = 0
    n = 0

    x_last = x_0
    j = 1
    ksi = np.random.rand(x_last.shape[0])
    ksi = np.array([i-0.5 for i in ksi])

    while True:

        x = x_last + alfa*ksi

        y_last = f(x_last)
        y = f(x)
        n_func += 2

        #print(x_last, y_last)
        #print(x, y)
        #print()

        if (y < y_last):
            x_last = x
        else:
            j += 1
            ksi = np.random.rand(x_last.shape[0])
            ksi = np.array([i-0.5 for i in ksi])

        if (j >= M):
            if (alfa < e):
                break
            else:
                alfa *= gamma
                j = 1
                
        n += 1

    return x_last, f(x_last), n, n_func

In [275]:
eps_global = 1e-6
alfa = 1.618
gamma = 0.618
M = 6
print("Случайного поиска\n")
x_min, y_min, count, func_count= random_find(func, alfa, gamma, M, np.array([4, 4]), eps_global)
print("(6)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = random_find(f_ovrag, alfa, gamma, M, np.array([0.5, 0.5]), eps_global)
print("(овражная)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = random_find(f_himelblay, alfa, gamma, M, np.array([-5, -2]), eps_global)
print("(Химельблау)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")
x_min, y_min, count, func_count = random_find(f_rozenbrok, alfa, gamma, M, np.array([-1, 1]), eps_global)
print("(Розенброка)\nx_min =", x_min, "\nf(x_min)", y_min, "\nИттераций =", count, "\nВызовов функции в совокопности", func_count, "\n")

Случайного поиска

(6)
x_min = [35.64367162 35.35626053] 
f(x_min) -2562.8883610428425 
Иттераций = 1181 
Вызовов функции в совокопности 2364 

(овражная)
x_min = [1.57885986e-07 7.74474424e-09] 
f(x_min) 2.4987965513714466e-14 
Иттераций = 196 
Вызовов функции в совокопности 394 

(Химельблау)
x_min = [-3.77931023 -3.28318601] 
f(x_min) 5.1629262042837454e-14 
Иттераций = 199 
Вызовов функции в совокопности 400 

(Розенброка)
x_min = [0.72548811 0.52378509] 
f(x_min) 0.07600596057393007 
Иттераций = 185384 
Вызовов функции в совокопности 370770 



## Задание 1

In [276]:
print("Овражная функция")
e_one = 1e-4
for i in [1, 250, 100]:
    for k in [1e-3, 1e-5]:
        global b
        b = i
        print("a =", b, "e =", k)
        t = PrettyTable(['Метод', 'x_min', 'y', 'Число иттераций', 'Вызовов функции'])

        x_0 = np.array([2, 2])

        x_min, y_min, count, func_count = fastest_descent(f_ovrag, grad_ovrag, x_0, k, e_one)
        t.add_row(['Наискорейший спуск', x_min, y_min, count, func_count])

        x_min, y_min, count, func_count = co_gradients(f_ovrag, grad_ovrag, x_0, k, e_one)
        t.add_row(['Сопряженных градиентов', x_min, y_min, count, func_count])
        
        x_min, y_min, count, func_count = newton(f_ovrag, grad_ovrag, H_1_ovrag, x_0, k)
        t.add_row(['Ньютона', x_min, y_min, count, func_count])

        l = 1e-2
        sigma = 0.618
        x_min, y_min, count, func_count = right_simplex(f_ovrag, x_0, l, sigma, k)
        t.add_row(['Правильного симплекса', x_min, y_min, count, func_count])

        x_min, y_min, count, func_count = cycle_coordinate_fall(f_ovrag, x_0, k, e_one)
        t.add_row(['Циклического покоординатного спуска', x_min, y_min, count, func_count])

        gamma = 1.618
        x_min, y_min, count, func_count = hook_jivs(f_ovrag, gamma, x_0, k)
        t.add_row(['Хука-Дживса', x_min, y_min, count, func_count])
        
        alfa = 1.618
        gamma = 0.618
        M = 12
        x_min, y_min, count, func_count = random_find(f_ovrag, alfa, gamma, M, x_0, k)
        t.add_row(['Случайного поиска', x_min, y_min, count, func_count])

        print(t)
        print('\n')

Овражная функция
a = 1 e = 0.001
+-------------------------------------+-----------------------------------+------------------------+-----------------+-----------------+
|                Метод                |               x_min               |           y            | Число иттераций | Вызовов функции |
+-------------------------------------+-----------------------------------+------------------------+-----------------+-----------------+
|          Наискорейший спуск         | [-2.91038305e-11 -2.91038305e-11] | 1.6940658945086007e-21 |        3        |       105       |
|        Сопряженных градиентов       |  [1.19151082e-07 1.19151082e-07]  | 2.8393960631051957e-14 |        2        |       111       |
|               Ньютона               |              [0. 0.]              |          0.0           |        1        |        12       |
|        Правильного симплекса        |     [ 0.00107767 -0.0059934 ]     | 3.7082198015087856e-05 |       659       |       3320      |
| Циклич

## Задание 2

In [277]:
print("Функция 6")
e_one = 1e-4
for k in [1e-3, 1e-6, 1e-8]:
    print("e =", k)
    t = PrettyTable(['Метод', 'x_min', 'y', 'Число иттераций', 'Вызовов функции'])
    x_0 = np.array([4, 4])

    x_min, y_min, count, func_count = fastest_descent(func, grad_func, x_0, k, e_one)
    t.add_row(['Наискорейший спуск', x_min, y_min, count, func_count])

    x_min, y_min, count, func_count = co_gradients(func, grad_func, x_0, k, e_one)
    t.add_row(['Сопряженных градиентов', x_min, y_min, count, func_count])
    
    x_min, y_min, count, func_count = newton(func, grad_func, H_1_func, x_0, k)
    t.add_row(['Ньютона', x_min, y_min, count, func_count])

    l = 1e-2
    sigma = 0.618
    x_min, y_min, count, func_count = right_simplex(func, x_0, l, sigma, k)
    t.add_row(['Правильного симплекса', x_min, y_min, count, func_count])

    x_min, y_min, count, func_count = cycle_coordinate_fall(func, x_0, k, e_one)
    t.add_row(['Циклического покоординатного спуска', x_min, y_min, count, func_count])

    gamma = 1.618
    x_min, y_min, count, func_count = hook_jivs(func, gamma, x_0, k)
    t.add_row(['Хука-Дживса', x_min, y_min, count, func_count])
    
    alfa = 1.618
    gamma = 0.618
    M = 12
    x_min, y_min, count, func_count = random_find(func, alfa, gamma, M, x_0, k)
    t.add_row(['Случайного поиска', x_min, y_min, count, func_count])

    print(t)
    print('\n')

Функция 6
e = 0.001
+-------------------------------------+---------------------------+---------------------+-----------------+-----------------+
|                Метод                |           x_min           |          y          | Число иттераций | Вызовов функции |
+-------------------------------------+---------------------------+---------------------+-----------------+-----------------+
|          Наискорейший спуск         | [35.64341503 35.35600207] |  -2562.888360874309 |       2380      |      56902      |
|        Сопряженных градиентов       | [35.64359131 35.35618129] |   -2562.8883610191  |        28       |       855       |
|               Ньютона               | [35.64370546 35.35629454] |  -2562.88836104516  |        1        |        12       |
|        Правильного симплекса        | [35.64265945 35.35775293] | -2562.8870406893725 |      10466      |      52803      |
| Циклического покоординатного спуска | [35.45385742 35.16699219] |  -2562.816420853138 |       57

## Задание 3

In [278]:
b = 250
e_one = 1e-4
for k in [1e-3]:
    print("a =", b, "e =", k)

    print("Овражная функция")
    t = PrettyTable(['Метод', 'x_min', 'y', 'Число иттераций', 'Вызовов функции'])
    x_0 = np.array([2, 2])

    x_min, y_min, count, func_count = fastest_descent(f_ovrag, grad_ovrag, x_0, k, e_one)
    t.add_row(['Наискорейший спуск', x_min, y_min, count, func_count])

    x_min, y_min, count, func_count = co_gradients(f_ovrag, grad_ovrag, x_0, k, e_one)
    t.add_row(['Сопряженных градиентов', x_min, y_min, count, func_count])
    
    x_min, y_min, count, func_count = newton(f_ovrag, grad_ovrag, H_1_ovrag, x_0, k)
    t.add_row(['Ньютона', x_min, y_min, count, func_count])

    l = 1e-2
    sigma = 0.618
    x_min, y_min, count, func_count = right_simplex(f_ovrag, x_0, l, sigma, k)
    t.add_row(['Правильного симплекса', x_min, y_min, count, func_count])

    x_min, y_min, count, func_count = cycle_coordinate_fall(f_ovrag, x_0, k, e_one)
    t.add_row(['Циклического покоординатного спуска', x_min, y_min, count, func_count])

    gamma = 1.618
    x_min, y_min, count, func_count = hook_jivs(f_ovrag, gamma, x_0, k)
    t.add_row(['Хука-Дживса', x_min, y_min, count, func_count])
    
    alfa = 1.618
    gamma = 0.618
    M = 12
    x_min, y_min, count, func_count = random_find(f_ovrag, alfa, gamma, M, x_0, k)
    t.add_row(['Случайного поиска', x_min, y_min, count, func_count])

    print(t)
    print('\n')
    

    print("Функция 6")
    t = PrettyTable(['Метод', 'x_min', 'y', 'Число иттераций', 'Вызовов функции'])
    x_0 = np.array([4, 4])

    x_min, y_min, count, func_count = fastest_descent(func, grad_func, x_0, k, e_one)
    t.add_row(['Наискорейший спуск', x_min, y_min, count, func_count])

    x_min, y_min, count, func_count = co_gradients(func, grad_func, x_0, k, e_one)
    t.add_row(['Сопряженных градиентов', x_min, y_min, count, func_count])
    
    x_min, y_min, count, func_count = newton(func, grad_func, H_1_func, x_0, k)
    t.add_row(['Ньютона', x_min, y_min, count, func_count])

    l = 1e-2
    sigma = 0.618
    x_min, y_min, count, func_count = right_simplex(func, x_0, l, sigma, l)
    t.add_row(['Правильного симплекса', x_min, y_min, count, func_count])

    x_min, y_min, count, func_count = cycle_coordinate_fall(func, x_0, k, e_one)
    t.add_row(['Циклического покоординатного спуска', x_min, y_min, count, func_count])

    gamma = 1.618
    x_min, y_min, count, func_count = hook_jivs(func, gamma, x_0, k)
    t.add_row(['Хука-Дживса', x_min, y_min, count, func_count])
    
    alfa = 1.618
    gamma = 0.618
    M = 12
    x_min, y_min, count, func_count = random_find(func, alfa, gamma, M, x_0, k)
    t.add_row(['Случайного поиска', x_min, y_min, count, func_count])

    print(t)
    print('\n')

a = 250 e = 0.001
Овражная функция
+-------------------------------------+-----------------------------------+------------------------+-----------------+-----------------+
|                Метод                |               x_min               |           y            | Число иттераций | Вызовов функции |
+-------------------------------------+-----------------------------------+------------------------+-----------------+-----------------+
|          Наискорейший спуск         |  [3.53998089e-04 1.49749139e-06]  | 1.2587526681517547e-07 |       1078      |      27939      |
|        Сопряженных градиентов       | [ 2.54911203e-04 -8.32213359e-07] | 6.515286601000418e-08  |        13       |       413       |
|               Ньютона               |              [0. 0.]              |          0.0           |        1        |        12       |
|        Правильного симплекса        | [ 1.27538822e-01 -2.54346821e-05] |  0.01626631277620474   |       1009      |       5236      |
| Цикл

## Задание 4

In [279]:
print("Функция Розенброка")
for i in [1e-4, 1e-6, 1e-8]:
    for k in [1e-3, 1e-5]:
        print("e =", k, "   e_one =", i)
        x_0 = np.array([-1, 1])

        t = PrettyTable(['Метод', 'x_min', 'y', 'Число иттераций', 'Вызовов функции'])

        x_min, y_min, count, func_count = fastest_descent(f_rozenbrok, grad_rozenbrok, x_0, k, i)
        t.add_row(['Наискорейший спуск', x_min, y_min, count, func_count])
        if not(k == 1e-5 and i == 1e-4):
            x_min, y_min, count, func_count = co_gradients(f_rozenbrok, grad_rozenbrok, x_0, k, i)
            t.add_row(['Сопряженных градиентов', x_min, y_min, count, func_count])
        
        x_min, y_min, count, func_count = newton(f_rozenbrok, grad_rozenbrok, H_1_rozenbrog, x_0, k)
        t.add_row(['Ньютона', x_min, y_min, count, func_count])

        l = 1e-2
        sigma = 0.618
        x_min, y_min, count, func_count = right_simplex(f_rozenbrok, x_0, l, sigma, k)
        t.add_row(['Правильного симплекса', x_min, y_min, count, func_count])

        x_min, y_min, count, func_count = cycle_coordinate_fall(f_rozenbrok, x_0, k, i)
        t.add_row(['Циклического покоординатного спуска', x_min, y_min, count, func_count])

        gamma = 1.618
        x_min, y_min, count, func_count = hook_jivs(f_rozenbrok, gamma, x_0, k)
        t.add_row(['Хука-Дживса', x_min, y_min, count, func_count])
        
        alfa = 1.618
        gamma = 0.618
        M = 12
        x_min, y_min, count, func_count = random_find(f_rozenbrok, alfa, gamma, M, x_0, k)
        t.add_row(['Случайного поиска', x_min, y_min, count, func_count])

        print(t)
        print('\n')
    

Функция Розенброка
e = 0.001    e_one = 0.0001
+-------------------------------------+-------------------------+------------------------+-----------------+-----------------+
|                Метод                |          x_min          |           y            | Число иттераций | Вызовов функции |
+-------------------------------------+-------------------------+------------------------+-----------------+-----------------+
|          Наискорейший спуск         | [1.00024238 1.0002433 ] | 5.892091357054967e-06  |        27       |       533       |
|        Сопряженных градиентов       | [1.00030107 1.00030179] |  9.11733150245321e-06  |        20       |       463       |
|               Ньютона               |        [ 1. -1.]        |   399.9999999999057    |        1        |        12       |
|        Правильного симплекса        | [0.73364454 0.53687652] |  0.07112959033000961   |       1579      |       8020      |
| Циклического покоординатного спуска | [0.99951172 0.9987793 ] 

## Задание 5

In [280]:
e_one = 1e-4
print("Функция Химельблау")
for x_0 in [np.array([0, 0]), np.array([-5, 0]), np.array([-5, -5])]:
    for k in [1e-3, 1e-6]:
        print("x_0 =", x_0,  "e =", k)

        t = PrettyTable(['Метод', 'x_min', 'y', 'Число иттераций', 'Вызовов функции'])

        x_min, y_min, count, func_count = fastest_descent(f_himelblay, grad_himelblay, x_0, k, e_one)
        t.add_row(['Наискорейший спуск', x_min, y_min, count, func_count])

        x_min, y_min, count, func_count = co_gradients(f_himelblay, grad_himelblay, x_0, k, e_one)
        t.add_row(['Сопряженных градиентов', x_min, y_min, count, func_count])

        x_min, y_min, count, func_count = newton(f_himelblay, grad_himelblay, H_1_himelblay, x_0, k)
        t.add_row(['Ньютона', x_min, y_min, count, func_count])

        print(t)
        print('\n')

Функция Химельблау
x_0 = [0 0] e = 0.001
+------------------------+---------------------------+-----------------------+-----------------+-----------------+
|         Метод          |           x_min           |           y           | Число иттераций | Вызовов функции |
+------------------------+---------------------------+-----------------------+-----------------+-----------------+
|   Наискорейший спуск   |  [3.00000064 2.00000502]  | 5.070742963083197e-10 |        12       |       348       |
| Сопряженных градиентов |  [3.0000118  2.00000247]  | 5.843453975029365e-09 |        8        |       265       |
|        Ньютона         | [-0.27084639 -0.92302828] |   181.6165215217085   |        3        |        32       |
+------------------------+---------------------------+-----------------------+-----------------+-----------------+


x_0 = [0 0] e = 1e-06
+------------------------+---------------------------+-----------------------+-----------------+-----------------+
|         Метод

![Alt text](image.png)